In [1]:
import torch
import torchvision
import h5py
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from ECE_CNN import CNN

C:\Users\Michael\Anaconda3\envs\ECE\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Load in the data
from load_data import EEGDataLoader
data = EEGDataLoader('project_datasets/')
X_train, y_train, X_test, y_test =  data.load_all_data()

(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)
(288, 22, 1000)


In [3]:
#Initialize the CNN
cnn = CNN()
cuda = False
if torch.cuda.is_available():
    print('Using GPU')
    cuda = True
    cnn.cuda()
    cnn = torch.nn.DataParallel(cnn,device_ids=range(torch.cuda.device_count()))
lr = 0.001
epochs = 1
batch_size = 100
optimizer = torch.optim.Adam(cnn.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()

Using GPU


In [4]:
#train the data 

X_train_i, y_train_i = X_train[0], y_train[0]
for epoch in range(10):
    print('Epoch: ',epoch)
    for i in range(X_train_i.shape[0]):
        image, label = X_train_i[i], y_train_i[i]
        if np.any(np.isnan(image)):
            continue
        if cuda:
            image = autograd.Variable(torch.cuda.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
            label = autograd.Variable(torch.cuda.LongTensor([int(label %769)]))
        else:
            image = autograd.Variable(torch.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
            label = autograd.Variable(torch.LongTensor([int(label %769)]))
        optimizer.zero_grad()
        scores = cnn(image)
        scores = torch.mean(scores,0)
        scores = scores.view(1,4)
        loss = criterion(scores,label)
        loss.backward()
        optimizer.step()
        
    print('Loss: ',loss.data[0])

Epoch:  0
Loss:  1.436803936958313
Epoch:  1
Loss:  1.2308070659637451
Epoch:  2
Loss:  1.4307893514633179
Epoch:  3
Loss:  1.770952582359314
Epoch:  4
Loss:  1.6367628574371338
Epoch:  5
Loss:  1.4858617782592773
Epoch:  6
Loss:  1.3417209386825562
Epoch:  7
Loss:  1.2900066375732422
Epoch:  8
Loss:  0.681967556476593
Epoch:  9
Loss:  0.44113689661026


In [5]:
# Training Accuracy
predictions = []
labels = []
X_train_i, y_train_i = X_train[0], y_train[0]
for i in range(X_train_i.shape[0]):
    image, label = X_train_i[i], y_train_i[i]
    if np.any(np.isnan(image)):
        continue
    if cuda:
        image = autograd.Variable(torch.cuda.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
        label = autograd.Variable(torch.cuda.LongTensor([int(label %769)]))
    else:
        image = autograd.Variable(torch.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
        label = autograd.Variable(torch.LongTensor([int(label %769)]))
    optimizer.zero_grad()
    scores = cnn(image)
    scores = torch.mean(scores,0)
    scores = scores.view(1,4)
    prediction = np.argmax(scores.data.cpu().numpy())
    predictions.append(prediction)
    labels.append(int(label%769))
diffs = [1 if prediction != label else 0 for prediction, label in zip(predictions, labels)]
error = sum(diffs)/len(diffs)
print("accuracy: {}".format(1 - error))

accuracy: 0.7974683544303798


In [6]:
# Testing Accuracy
X_test_i, y_test_i = X_test[0], y_test[0]
predictions = []
labels = []
for i in range(X_test_i.shape[0]):
    image, label = X_test_i[i], y_test_i[i]
    if np.any(np.isnan(image)):
        continue
    if cuda:
        image = autograd.Variable(torch.cuda.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
        label = autograd.Variable(torch.cuda.LongTensor([int(label %769)]))
    else:
        image = autograd.Variable(torch.FloatTensor(image.reshape((image.shape[0],1,image.shape[1]))))
        label = autograd.Variable(torch.LongTensor([int(label %769)]))
    optimizer.zero_grad()
    scores = cnn(image)
    scores = torch.mean(scores,0)
    scores = scores.view(1,4)
    prediction = np.argmax(scores.data.cpu().numpy())
    predictions.append(prediction)
    labels.append(int(label%769))


diffs = [1 if prediction != label else 0 for prediction, label in zip(predictions, labels)]
error = sum(diffs)/len(diffs)
print("accuracy: {}".format(1 - error))

accuracy: 0.36
